<a href="https://colab.research.google.com/github/PlasticRosewood/COP4331-P1-Group2/blob/main/Music_Genre_RNN_vRNN_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Libraries

In [25]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
import pandas as pd

Parse Music Data

In [26]:
# Import CSV File
df = pd.read_csv("features_3_sec.csv")

# Seperate columns into features
chroma_mean = df["chroma_stft_mean"].to_numpy()
chroma_var = df["chroma_stft_var"].to_numpy()
rms_mean = df["rms_mean"].to_numpy()
rms_var = df["rms_var"].to_numpy()
scm = df["spectral_centroid_mean"].to_numpy()
scv = df["spectral_centroid_var"].to_numpy()
sbm = df["spectral_bandwidth_mean"].to_numpy()
sbv = df["spectral_bandwidth_var"].to_numpy()

# Convert label from string to int
map = {}
y = []
for label in df['label']:
  if label not in map:
    map.update({label: len(map)})
  y.append(map.get(label))

Normalize Music Data

In [30]:
features = [chroma_mean, chroma_var, rms_mean, rms_var, scm, scv, sbm, sbv]

# normalize between -1 and 1
def normalize_range(arr):
  arr_min = arr.min()
  arr_max = arr.max()
  for i in range(len(arr)):
    arr[i] = (2 * (arr[i] - arr_min) / (arr_max - arr_min)) -1

# normalize all data
for x in features:
  normalize_range(x)

Set up RNN

Initializing parameters and weight

In [50]:
input_size = 8 # number of features
hidden_size = 5 # (2/3 input size) + output size
output_size = 2 # number of genres to detect (subject to change)

# INITIALIZE WEIGHTS
# initalize weights for inputs to hidden layer
u = np.random.rand(input_size, hidden_size)

# initialize weights for hidden layer to hidden layer
w = np.random.rand(hidden_size, hidden_size)

# initialize weights for hidden layer to output layer
v = np.random.rand(output_size, hidden_size)

# INITIALIZE BIASES
# initialize bias for inputs to hidden layer
b_ih = np.random.rand(hidden_size)

# initialize bias for hidden layer to hidden layer
b_hh = np.random.rand(hidden_size)

# initialize bias for hidden layer to output
b_ho = np.random.rand(output_size)

Get all features from a sample

In [39]:
"""
Given an index of test data, return all features for that sample
"""
def get_features(sample_index):
  return_arr = [x[sample_index] for x in features]
  return return_arr

print(get_features(0))


[-0.28920239345338317, 0.43351398236789285, -0.4137334798119917, -0.784090682893563, -0.47565414285940366, -0.9304322634294081, -0.08159019673048362, -0.8117405004376876]


Recurrence Neural Network (Forward)

In [63]:
"""
This function does 1 iteration of a recurrence neural network training model
it takes in two arguments:
   mem_h:
     The current state of the hidden layer either calculated in previous rounds
       or defined as h_-1

   features_x:
     The features of the current audio sample. This is assumed to be from the
       list of 8 features defined earlier. This will be used in calculating the
       hidden layer state.
"""
def RNN(mem_h, features_x):
  # update state of memory
  mem_h = np.add(
      np.add(np.dot(w, mem_h), b_hh),
      np.add(np.dot(features_x, u), b_ih)
  )

  # calculate output Y
  # this will only be used on the final iteration of RNN
  output_y = np.add(
      np.dot(v, mem_h),
      b_ho
  )

  return mem_h, output_y


RNN Wrapper Function

In [70]:
"""
This function takes in a number of rounds to perform (num_rounds)
and runs that many rounds, keeping track of the output and hidden layer state
after each round. It returns these two arrays
"""
def RNN_Wrapper(num_rounds, target):
  # initialize hidden memory with identity matrix
  init_mem_h = np.ones(hidden_size)

  # define containers for storing state
  hidden_states = np.zeros((num_rounds + 1, hidden_size))
  hidden_states[0] = init_mem_h # initialize with filler value

  outputs = np.zeros((num_rounds, output_size))

  target_features = get_features(target)

  # loop through each round
  for i in range(num_rounds):
    print('Round: ' + str(i))
    hidden_states[i + 1], outputs[i] = RNN(hidden_states[i], target_features)
    print('output: ', outputs[i])
    print('hidden state: ', hidden_states[i+1])

  return hidden_states, outputs